In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
from datetime import datetime
import time
import random
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime

In [2]:
chrome_options = webdriver.ChromeOptions()
opgg = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
url = "https://www.op.gg/"
opgg.get(url)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [/Users/myceleb_39/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [3]:
opgg.find_element_by_xpath('//*[@id="searchUserName"]').send_keys('뱀꼬치')
opgg.find_element_by_xpath('//*[@id="summoner-search-form"]/button[1]/i').click()
opgg.find_element_by_xpath('//*[@id="SummonerRefreshButton"]').click()

In [4]:
while True:
    try:
        opgg.find_element_by_css_selector('div.GameMoreButton.Box').click() 
        
        time.sleep(1)
            
    except:
        break

In [5]:
source = opgg.page_source
soup = BeautifulSoup(source, 'lxml')

In [6]:
num = 1
num_list = []
while (num*2)+3 < 86:
    try:
        a = (num*2)+3
        num_list.append(int(a))
        num +=1 
    
    except:
        print("end")
        break

In [7]:
df = pd.DataFrame(data = None, columns = ('game_num', 'play_date', 'game_type', 'result', 'game_length', 'champion',
                                          'spell_d', 'spell_f', 'level', 'creep_score','cs_1m', 'kill', 'death',
                                          'assist', 'kda', 'kill_ratio', 'average_tier'))

In [11]:
for i in tqdm(num_list):
    try:
        a = soup.select(f'#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div:nth-child({i})')
        b = a[0].select('div[class="GameItemWrap"]')
    except:
        pass
    for j in range(len(b)):
        try:
            game_num = b[j].select('div')[0]['data-game-id']
            play_date = int(b[j].select_one(('div > span'))['data-datetime'])
        #     play_date = str(datetime.datetime.fromtimestamp(play_date)).split(' ')[0]
            game_type = b[j].select('div[class="GameType"]')[0]['title']
            result = b[j].select('div[class="GameResult"]')[0].text.replace('\t', '').replace('\n', '')
            game_length = b[j].select('div[class="GameLength"]')[0].text
            champion = b[j].select('div[class="ChampionName"]')[0].text.replace('\n', '')
            spell_d = b[j].select('div[class="SummonerSpell"]')[0].select('img')[0]['alt']
            spell_f = b[j].select('div[class="SummonerSpell"]')[0].select('img')[1]['alt']
            level = b[j].select('div[class="Level"]')[0].text.replace('\t', '').replace('\n', '')[2:]
            creep_score = b[j].select('span[class="CS tip"]')[0].text.split(' ')[0]
            cs_1m = b[j].select('span[class="CS tip"]')[0].text.split(' ')[1][1:-1]
            kill = int(b[j].select('span[class="Kill"]')[0].text)
            death = int(b[j].select('span[class="Death"]')[0].text)
            assist = int(b[j].select('span[class="Assist"]')[0].text)
            kda = round((kill + assist) / death, 1)
            kill_ratio = b[j].select('div[class="CKRate tip"]')[0].text.replace('\t', '').replace('\n', '').split(' ')[1][:-1]

            if death == 0:
                kda = 'perfect'

            else:
                kda = round((kill + assist) / death, 1)

            if b[j].select('div[class="MMR"]') :
                average_tier = b[j].select('div[class="MMR"]')[0].select('b')[0].text

            print(game_num, '/', play_date, '/', game_type, '/', result, '/', game_length, '/', champion, '/', spell_d,
                  '/', spell_f, '/', level, '/', creep_score, '/', cs_1m, '/', kill, '/', death, '/', assist, '/', kda, '/',
                  kill_ratio, '/', average_tier)

            data_list = [game_num, play_date, game_type, result, game_length, champion, spell_d, spell_f,
                         level, creep_score,cs_1m, kill, death, assist, kda, kill_ratio, average_tier]
            df = df.append(pd.Series(data_list, index=df.columns), ignore_index=True)

        except:
            print('fail')
            continue

  0%|          | 0/41 [00:00<?, ?it/s]

5645119577 / 1640272200 / 솔랭 / 승리 / 22분 52초 / 라이즈 / 점멸 / 순간이동 / 13 / 140 / 6.1 / 4 / 4 / 5 / 2.2 / 23 / Silver 3
fail
5644935356 / 1640269581 / 솔랭 / 승리 / 40분 51초 / 퀸 / 점멸 / 유체화 / 18 / 211 / 5.2 / 10 / 9 / 10 / 2.2 / 53 / Silver 3
5641110851 / 1640094201 / 솔랭 / 승리 / 22분 50초 / 라이즈 / 점멸 / 순간이동 / 13 / 132 / 5.8 / 3 / 6 / 3 / 1.0 / 30 / Silver 1
5641072072 / 1640092528 / 솔랭 / 승리 / 28분 38초 / 그웬 / 점멸 / 순간이동 / 15 / 209 / 7.3 / 3 / 5 / 2 / 1.0 / 12 / Silver 2
5624417917 / 1639407112 / 자유 5:5 랭크 / 패배 / 29분 2초 / 라이즈 / 점멸 / 순간이동 / 14 / 174 / 6 / 5 / 4 / 9 / 3.5 / 39 / Gold 3
5622703268 / 1639321904 / 무작위 총력전 / 승리 / 25분 42초 / 애니비아 / 점멸 / 회복 / 18 / 63 / 2.5 / 10 / 7 / 35 / 6.4 / 56 / Gold 3
5620712204 / 1639245930 / 솔랭 / 승리 / 27분 36초 / 카이사 / 점멸 / 회복 / 14 / 185 / 6.7 / 4 / 4 / 12 / 4.0 / 40 / Silver 2
5620597544 / 1639243898 / 솔랭 / 패배 / 29분 21초 / 징크스 / 점멸 / 회복 / 12 / 185 / 6.3 / 2 / 5 / 7 / 1.8 / 38 / Silver 2
5620641771 / 1639241875 / 솔랭 / 패배 / 23분 57초 / 칼리스타 / 점멸 / 회복 / 12 / 161 / 6.7 / 7 / 6 / 2 /

  2%|▏         | 1/41 [00:02<01:21,  2.05s/it]

5618131947 / 1639157429 / 무작위 총력전 / 패배 / 35분 39초 / 아우렐리온 솔 / 점멸 / 유체화 / 18 / 157 / 4.4 / 23 / 19 / 38 / 3.2 / 56 / Silver 2
5618048771 / 1639155144 / 무작위 총력전 / 패배 / 17분 12초 / 베이가 / 점멸 / 방어막 / 15 / 51 / 3 / 5 / 8 / 7 / 1.5 / 60 / Silver 2
5617975075 / 1639153922 / 솔랭 / 승리 / 20분 25초 / 벡스 / 점멸 / 점화 / 12 / 127 / 6.2 / 9 / 1 / 1 / 10.0 / 40 / Gold 4
5617990426 / 1639152242 / 솔랭 / 패배 / 15분 37초 / 코그모 / 점멸 / 회복 / 9 / 86 / 5.5 / 1 / 2 / 0 / 0.5 / 25 / Silver 1
5617912148 / 1639150907 / 솔랭 / 승리 / 30분 20초 / 베인 / 점멸 / 회복 / 16 / 179 / 5.9 / 9 / 3 / 8 / 5.7 / 44 / Silver 1
5617826920 / 1639148690 / 궁극기 모드 / 패배 / 18분 5초 / 아칼리 / 점멸 / 게임 시작 후 결정 / 11 / 53 / 2.9 / 4 / 8 / 1 / 0.6 / 31 / Silver 1
5611854452 / 1638887773 / 솔랭 / 승리 / 35분 37초 / 모르가나 / 점멸 / 점화 / 14 / 33 / 0.9 / 0 / 7 / 13 / 1.9 / 45 / Silver 1
5611814577 / 1638885324 / 솔랭 / 패배 / 28분 58초 / 징크스 / 점멸 / 회복 / 13 / 161 / 5.6 / 4 / 8 / 10 / 1.8 / 37 / Silver 1
5611669617 / 1638883304 / 솔랭 / 승리 / 29분 51초 / 징크스 / 점멸 / 회복 / 15 / 207 / 6.9 / 17 / 5 / 1

  5%|▍         | 2/41 [00:03<01:12,  1.85s/it]

...

 73%|███████▎  | 30/41 [00:40<00:13,  1.27s/it]

5358559254 / 1627622590 / 솔랭 / 패배 / 19분 15초 / 라이즈 / 점멸 / 순간이동 / 12 / 130 / 6.8 / 1 / 5 / 2 / 0.6 / 27 / Silver 3
5355980415 / 1627490840 / 자유 5:5 랭크 / 패배 / 38분 25초 / 아크샨 / 점멸 / 순간이동 / 18 / 260 / 6.8 / 4 / 7 / 5 / 1.3 / 28 / Silver 2
5355118794 / 1627476933 / 일반 / 패배 / 25분 4초 / 아크샨 / 점멸 / 방어막 / 13 / 161 / 6.4 / 5 / 3 / 6 / 3.7 / 69 / Silver 2
5355162050 / 1627475231 / 일반 / 패배 / 26분 2초 / 아크샨 / 점멸 / 순간이동 / 13 / 130 / 5 / 9 / 11 / 2 / 1.0 / 44 / Silver 2
5355045982 / 1627473473 / 일반 / 패배 / 27분 28초 / 아크샨 / 점멸 / 회복 / 12 / 120 / 4.4 / 7 / 8 / 5 / 1.5 / 71 / Silver 2
5355051881 / 1627471629 / 일반 / 승리 / 19분 14초 / 아크샨 / 점멸 / 회복 / 10 / 96 / 5 / 3 / 2 / 3 / 3.0 / 26 / Silver 2
5354974387 / 1627468880 / 일반 / 패배 / 20분 18초 / 아크샨 / 점멸 / 순간이동 / 12 / 129 / 6.4 / 4 / 6 / 3 / 1.2 / 41 / Silver 2
5354773217 / 1627459746 / 솔랭 / 승리 / 16분 52초 / 라이즈 / 점멸 / 순간이동 / 11 / 95 / 5.6 / 0 / 1 / 5 / 5.0 / 28 / Silver 4
5352829120 / 1627381511 / 솔랭 / 패배 / 27분 0초 / 케이틀린 / 점멸 / 회복 / 12 / 153 / 5.7 / 3 / 7 / 10 / 1.9 / 59 

 76%|███████▌  | 31/41 [00:41<00:11,  1.15s/it]

5352863813 / 1627379408 / 솔랭 / 패배 / 25분 43초 / 베인 / 점멸 / 회복 / 12 / 151 / 5.9 / 6 / 7 / 6 / 1.7 / 43 / Silver 2
5352639619 / 1627377507 / 솔랭 / 패배 / 25분 25초 / 룰루 / 점멸 / 점화 / 11 / 23 / 0.9 / 3 / 7 / 5 / 1.1 / 33 / Silver 2
fail
5352640107 / 1627373892 / 솔랭 / 승리 / 29분 43초 / 사일러스 / 점멸 / 순간이동 / 15 / 111 / 3.7 / 6 / 4 / 8 / 3.5 / 47 / Silver 2


100%|██████████| 41/41 [00:48<00:00,  1.18s/it]


In [12]:
df

,game_num,play_date,game_type,result,game_length,champion,spell_d,spell_f,level,creep_score,cs_1m,kill,death,assist,kda,kill_ratio,average_tier
0,5645119577,1640272200,솔랭,승리,22분 52초,라이즈,점멸,순간이동,13,140,6.1,4,4,5,2.2,23,Silver 3
1,5644935356,1640269581,솔랭,승리,40분 51초,퀸,점멸,유체화,18,211,5.2,10,9,10,2.2,53,Silver 3
2,5641110851,1640094201,솔랭,승리,22분 50초,라이즈,점멸,순간이동,13,132,5.8,3,6,3,1.0,30,Silver 1
3,5641072072,1640092528,솔랭,승리,28분 38초,그웬,점멸,순간이동,15,209,7.3,3,5,2,1.0,12,Silver 2
4,5624417917,1639407112,자유 5:5 랭크,패배,29분 2초,라이즈,점멸,순간이동,14,174,6,5,4,9,3.5,39,Gold 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,5354773217,1627459746,솔랭,승리,16분 52초,라이즈,점멸,순간이동,11,95,5.6,0,1,5,5.0,28,Silver 4
574,5352829120,1627381511,솔랭,패배,27분 0초,케이틀린,점멸,회복,12,153,5.7,3,7,10,1.9,59,Silver 2
575,5352863813,1627379408,솔랭,패배,25분 43초,베인,점멸,회복,12,151,5.9,6,7,6,1.7,43,Silver 2
576,5352639619,1627377507,솔랭,패배,25분 25초,룰루,점멸,점화,11,23,0.9,3,7,5,1.1,33,Silver 2


In [13]:
numnum = list(df.game_num)
datedate = list(df.play_date)

In [14]:
print(len(numnum))
print(len(datedate))

578
578


In [15]:
df1 = pd.DataFrame(data = None, columns = ('game_num', 'op_score', 'damage_to_cham', 'total_damage',
                                           'pink_ward', 'green_ward', 'del_ward', 'total_kill',
                                           'total_gold', 'get_baron', 'get_dragon', 'get_tower',
                                           'lose_baron', 'lose_dragon', 'lose_tower'))

In [16]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
opgg = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

for i, j in tqdm(zip(numnum, datedate)):
    detail_url = f'https://www.op.gg/summoner/matches/ajax/detail/gameId={i}&summonerId=3170632&gameTime={j}'
    opgg.get(detail_url)
    source = opgg.page_source
    soup = BeautifulSoup(source, 'lxml')
    
    a = soup.select('body > div > div.MatchDetailContent.tabItems > div > div')
    try:
        op_score = op_score = a[0].select('tr.Row.isRequester')[0].select('div[class="OPScore Text"]')[0].text
        damage_to_cham = a[0].select('tr.Row.isRequester')[0].select('td[class="Damage Cell tip"]')[0]['title'].replace('<br>', ': ').split(': ')[1]
        total_damage = a[0].select('tr.Row.isRequester')[0].select('td[class="Damage Cell tip"]')[0]['title'].replace('<br>', ': ').split(': ')[3]
        pink_ward = re.findall(r'\d+', a[0].select('tr.Row.isRequester')[0].select('td[class="Ward Cell tip"]')[0]['title'])[0]
        green_ward = re.findall(r'\d+', a[0].select('tr.Row.isRequester')[0].select('td[class="Ward Cell tip"]')[0]['title'])[1]
        del_ward = re.findall(r'\d+', a[0].select('tr.Row.isRequester')[0].select('td[class="Ward Cell tip"]')[0]['title'])[2]
        total_kill = a[0].select('div[class="text graph--data graph--data__left"]')[0].text
        total_gold = a[0].select('div[class="text graph--data graph--data__left"]')[1].text
        get_baron = a[0].select('div[class="ObjectScore"]')[0].text.replace('\n', '').replace('\t', '')
        get_dragon = a[0].select('div[class="ObjectScore"]')[1].text.replace('\n', '').replace('\t', '')
        get_tower = a[0].select('div[class="ObjectScore"]')[2].text.replace('\n', '').replace('\t', '')
        lose_baron = a[0].select('div[class="ObjectScore"]')[3].text.replace('\n', '').replace('\t', '')
        lose_dragon = a[0].select('div[class="ObjectScore"]')[4].text.replace('\n', '').replace('\t', '')
        lose_tower = a[0].select('div[class="ObjectScore"]')[5].text.replace('\n', '').replace('\t', '')
        
        print(i, '/', op_score, '/', damage_to_cham, '/', total_damage, '/', pink_ward, '/',
              green_ward, '/', del_ward, '/', total_kill, '/', total_gold, '/', get_baron, '/',
              get_dragon, '/', get_tower, '/', lose_baron, '/', lose_dragon, '/', lose_tower)

        data_list = [i, op_score, damage_to_cham, total_damage, pink_ward, green_ward, del_ward,
                     total_kill, total_gold, get_baron, get_dragon, get_tower, lose_baron,
                     lose_dragon, lose_tower]
        
        df1 = df1.append(pd.Series(data_list, index=df1.columns), ignore_index=True)
        
    except:
        print('error')
        continue



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [/Users/myceleb_39/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
1it [00:00,  1.45it/s]

5645119577 / 6.5 / 9,377 / 93,639 / 2 / 4 / 1 / 39 / 50331 / 0 / 3 / 11 / 0 / 0 / 1


2it [00:01,  1.85it/s]

5644935356 / 6.6 / 33,373 / 188,776 / 2 / 2 / 5 / 38 / 79208 / 2 / 2 / 9 / 1 / 4 / 10


3it [00:01,  2.17it/s]

...

5352863813 / 5.8 / 14,175 / 89,423 / 1 / 7 / 3 / 28 / 48741 / 0 / 1 / 5 / 0 / 2 / 9


577it [02:48,  2.88it/s]

5352639619 / 5.2 / 5,846 / 24,610 / 2 / 12 / 4 / 24 / 46655 / 0 / 0 / 6 / 1 / 3 / 7


578it [02:48,  3.43it/s]

5352640107 / 5.1 / 12,166 / 83,345 / 2 / 2 / 0 / 30 / 58637 / 2 / 3 / 10 / 0 / 1 / 2


In [17]:
df1

,game_num,op_score,damage_to_cham,total_damage,pink_ward,green_ward,del_ward,total_kill,total_gold,get_baron,get_dragon,get_tower,lose_baron,lose_dragon,lose_tower
0,5645119577,6.5,"9,377","93,639",2,4,1,39,50331,0,3,11,0,0,1
1,5644935356,6.6,"33,373","188,776",2,2,5,38,79208,2,2,9,1,4,10
2,5641110851,5.2,"10,323","100,788",0,5,0,20,40155,0,2,3,0,0,2
3,5641072072,5.9,"11,354","137,228",1,3,1,43,61856,1,3,11,0,1,2
4,5624417917,6.5,"22,522","133,223",1,4,5,36,53526,0,2,3,1,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,5354773217,7.7,"10,787","53,026",2,4,1,18,30585,0,1,3,0,0,0
471,5352829120,6.1,"13,966","122,076",2,6,5,22,48270,0,3,6,1,1,7
472,5352863813,5.8,"14,175","89,423",1,7,3,28,48741,0,1,5,0,2,9
473,5352639619,5.2,"5,846","24,610",2,12,4,24,46655,0,0,6,1,3,7


In [19]:
print(len(df), len(df1))

578 475


In [20]:
df2 = pd.merge(df, df1, left_on='game_num', right_on='game_num', how='inner')
df2

,game_num,play_date,game_type,result,game_length,champion,spell_d,spell_f,level,creep_score,...,green_ward,del_ward,total_kill,total_gold,get_baron,get_dragon,get_tower,lose_baron,lose_dragon,lose_tower
0,5645119577,1640272200,솔랭,승리,22분 52초,라이즈,점멸,순간이동,13,140,...,4,1,39,50331,0,3,11,0,0,1
1,5644935356,1640269581,솔랭,승리,40분 51초,퀸,점멸,유체화,18,211,...,2,5,38,79208,2,2,9,1,4,10
2,5641110851,1640094201,솔랭,승리,22분 50초,라이즈,점멸,순간이동,13,132,...,5,0,20,40155,0,2,3,0,0,2
3,5641072072,1640092528,솔랭,승리,28분 38초,그웬,점멸,순간이동,15,209,...,3,1,43,61856,1,3,11,0,1,2
4,5624417917,1639407112,자유 5:5 랭크,패배,29분 2초,라이즈,점멸,순간이동,14,174,...,4,5,36,53526,0,2,3,1,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,5354773217,1627459746,솔랭,승리,16분 52초,라이즈,점멸,순간이동,11,95,...,4,1,18,30585,0,1,3,0,0,0
471,5352829120,1627381511,솔랭,패배,27분 0초,케이틀린,점멸,회복,12,153,...,6,5,22,48270,0,3,6,1,1,7
472,5352863813,1627379408,솔랭,패배,25분 43초,베인,점멸,회복,12,151,...,7,3,28,48741,0,1,5,0,2,9
473,5352639619,1627377507,솔랭,패배,25분 25초,룰루,점멸,점화,11,23,...,12,4,24,46655,0,0,6,1,3,7


In [21]:
len(df2)

475

In [24]:
df2.to_excel("data.xlsx", index=False, encoding="utf-8-sig")